In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model

In [2]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [3]:
#Surface data
t2mData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_tas_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')
rhData  = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_hurs_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')
u10Data = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_uas_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')
v10Data = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_vas_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')

In [4]:
# Level Data
tLevData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_ta_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')
zLevData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_zg_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')
wLevData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_wap_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')
uLevData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/Regrid_NH_ua_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc')

In [5]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

#Calculate inv
inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [6]:
# AO data
AOData = xr.open_dataset('IPSL-CM6A-LR-AOindex-NDJF-Daily-1980-2014.nc')
aoTS=AOData.AO
#datetimeindex = aoTS.indexes['time'].to_datetimeindex()
#datetimeindex
#aoTS['time'] = datetimeindex
Darray=np.zeros((AOData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
for t in range(aoTS.time.shape[0]) :
    Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
# EU data
EUData = xr.open_dataset('IPSL-CM6A-LR-EUindex-NDJF-Daily-1980-2014.nc')
EUData.EU
euTS=EUData.EU
#datetimeindex = euTS.indexes['time'].to_datetimeindex()
#datetimeindex
#euTS['time'] = datetimeindex
Darray=np.zeros((EUData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
for t in range(euTS.time.shape[0]) :
    Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})

In [7]:
# create mask
oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [8]:
#AO
AO5D=AOData.AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

#EU
EU5D=EUData.EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

In [9]:
t1=AO5DAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
AO5DnormIpslCm6aLr = StandardScaler().fit(t1)
dump(AO5DnormIpslCm6aLr,'AO5DnormIpslCm6aLr.joblib')
# transform training data
t1.values = AO5DnormIpslCm6aLr.transform(t1)
AO5DAll.values=t1.unstack()

t1=EU5DAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
EU5DnormIpslCm6aLr = StandardScaler().fit(t1)
dump(EU5DnormIpslCm6aLr,'EU5DnormIpslCm6aLr.joblib')
# transform training data
t1.values = EU5DnormIpslCm6aLr.transform(t1)
EU5DAll.values=t1.unstack()

AO5DAll.values=AO5DAll.values*mask
AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

EU5DAll.values=EU5DAll.values*mask
EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)

In [10]:
t2m=t2mData.tas.shift(time=1)
ws=ws_ds.ws.shift(time=1)
rh=rh_ds.rh.shift(time=1)
inv=inv_ds.inv.shift(time=1)
w=wLevData.wap.sel(plev=70000,method='nearest').shift(time=1)
ushear=ushear_ds.ushear.shift(time=1)

In [11]:
t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))

In [12]:
t1=t2mTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
t2mnormIpslCm6aLr = StandardScaler().fit(t1)
dump(t2mnormIpslCm6aLr,'t2mnormIpslCm6aLr.joblib')
# transform training data
t1.values = t2mnormIpslCm6aLr.transform(t1)
t2mTsAll.values=t1.unstack()

t1=wsTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
wsnormIpslCm6aLr = StandardScaler().fit(t1)
dump(wsnormIpslCm6aLr,'wsnormIpslCm6aLr.joblib')
# transform training data
t1.values = wsnormIpslCm6aLr.transform(t1)
wsTsAll.values=t1.unstack()

t1=rhTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
rhnormIpslCm6aLr = StandardScaler().fit(t1)
dump(rhnormIpslCm6aLr,'rhnormIpslCm6aLr.joblib')
# transform training data
t1.values = rhnormIpslCm6aLr.transform(t1)
rhTsAll.values=t1.unstack()

t1=invTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
invnormIpslCm6aLr = StandardScaler().fit(t1)
dump(invnormIpslCm6aLr,'invnormIpslCm6aLr.joblib')
# transform training data
t1.values = invnormIpslCm6aLr.transform(t1)
invTsAll.values=t1.unstack()

t1=ushearTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
ushearnormIpslCm6aLr = StandardScaler().fit(t1)
dump(ushearnormIpslCm6aLr,'ushearnormIpslCm6aLr.joblib')
# transform training data
t1.values = ushearnormIpslCm6aLr.transform(t1)
ushearTsAll.values=t1.unstack()


t1=wTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
wnormIpslCm6aLr = StandardScaler().fit(t1)
dump(wnormIpslCm6aLr,'wnormIpslCm6aLr.joblib')
# transform training data
t1.values = wnormIpslCm6aLr.transform(t1)
wTsAll.values=t1.unstack()

/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)
/home/cccr/diptih/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cccr

In [13]:
t2mTsAll.values=t2mTsAll.values*mask
wsTsAll.values=wsTsAll.values*mask
rhTsAll.values=rhTsAll.values*mask
invTsAll.values=invTsAll.values*mask
ushearTsAll.values=ushearTsAll.values*mask
wTsAll.values=wTsAll.values*mask


In [14]:
t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)


In [15]:
t2mt=t2mTsAll.values
t2mt=t2mt[:,:,:,None]
t2mt.shape


wst=wsTsAll.values
wst=wst[:,:,:,None]
wst.shape

rht=rhTsAll.values
rht=rht[:,:,:,None]
rht.shape


invt=invTsAll.values
invt=invt[:,:,:,None]
invt.shape

wt=wTsAll.values
wt=wt[:,:,:,None]
wt.shape

usheart=ushearTsAll.values
usheart=usheart[:,:,:,None]
usheart.shape

aot=AO5DAll.values
aot=aot[:,:,:,None]
aot.shape

eut=EU5DAll.values
eut=eut[:,:,:,None]
eut.shape


(2170, 18, 26, 1)

In [16]:
X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
X.shape

(8, 2170, 18, 26, 1)

In [17]:
X_reshape = np.einsum('lkija->klija',X)
X_reshape.shape

(2170, 8, 18, 26, 1)

In [18]:
# Load saved model
# load model
model = load_model('../../March2021/Observation_models/modelCNN.h5')
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 8, 18, 26, 16)     448       
_________________________________________________________________
average_pooling3d (AveragePo (None, 4, 9, 13, 16)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 4, 9, 13, 32)      13856     
_________________________________________________________________
average_pooling3d_1 (Average (None, 2, 5, 7, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 5, 7, 32)       0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 2, 5, 7, 64)       55360     
_________________________________________________________________
average_pooling3d_2 (Average (None, 1, 3, 4, 64)       0

In [19]:
yLR=model.predict(X_reshape)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})

In [20]:
#dump(y_predLin_ds.yLR,'../Model_plots/IPSL-CM6A-LR-CNN-Y.joblib')
dump(y_predLin_ds.yLR,'../Model_plots/IPSL-CM6A-LR-CNN-Y1.joblib')

['../Model_plots/IPSL-CM6A-LR-CNN-Y1.joblib']